In [30]:
import linecache
from symspellpy import SymSpell, Verbosity
from rich import print
import inspect
import os
from collections import Counter
import re
import textstat
textstat.set_lang('es')

In [31]:
CONFIG_DIR = '/home/silvio/miniconda3/envs/classy3/prg/config/'
STOPWORD_ES = 'stopwords_es.txt'
STOPWORD_RED = 'stopwords_reddit.txt'
DICTIONARY = 'new_dic.txt'

dictionary = os.path.join(CONFIG_DIR, DICTIONARY)
stopwords_files = [os.path.join(CONFIG_DIR, STOPWORD_ES),
                  os.path.join(CONFIG_DIR,STOPWORD_RED)]
# stop_words_es = os.path.join(CONFIG_DIR, STOPWORD_ES)
# stop_words_red = os.path.join(CONFIG_DIR,STOPWORD_RED)


In [32]:
from collections import Counter

def read_all_stopwords(files=stopwords_files):
    # Initialize an empty Counter
    stopwords = Counter()

    # Iterate through the list of file paths
    for file_path in files:
        try:
            # Open and read the file
            with open(file_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
                # Update the Counter with words (stripped of whitespace and newline characters)
                for line in lines:
                    word = line.strip()
                    if word:
                        stopwords.update([word.lower()])
        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except Exception as e:
            print(f"An error occurred while reading {file_path}: {str(e)}")

    return stopwords


In [33]:
stopwords = read_all_stopwords()
print(len(stopwords))

710

In [21]:
s = """
Hace 2 años conocí a una chica por medio de un amigo, hablamos y tiempo después nos hicimos novios.
  
La relación era algo estable, teníamos defectos como cada persona, pero a diferencia yo los traba de resolver y cambiar mi actitud, mientras ella solo se dedicaba a enojarse y desquitarse conmigo aun cuando yo no tenía nada que ver, se enojaba con su familia y se desquitaba conmigo, se enojaba en la universidad con sus compañeros o amigos y se desquitaba conmigo, le explique que eso me hacía sentir mal y en cierto modo me hacía sentir inseguro, traté que en el tiempo de relación ella cambiara pero nunca lo hizo, y siempre me decía que era su actitud y debía aguantarle todos sus berrinches, pero cuando llegamos al año y 4 meses las cosas se volvieron densas, teníamos problemas, discusiones, peleas, etc. En mi mente pasaba que ya terminó la etapa de ilusión o enamoramiento como lo llaman los demás, el hecho es que ella buscaba que yo le solucione los problemas y si no lo hacía, la historia de siempre, peleas.
  
Al año y 6 meses tuvimos una discusión fuerte a la cual ella decidió terminar conmigo, ¿la causa fue que no le daba soluciones a sus problemas, como quería que solucione sus problemas si no podía resolver los que yo tenía? ¿Terminó conmigo haciéndome sentir que fui el malo de la relación, aun cuando yo estuve ahi cuando todos le dieron la espalda, la apoyé aun cuando su madre la trataba mal, enserio fui el malo de la relación? Me enfadé mucho con ella pero no le dije nada, simplemente me ahorré la discusión, tiempo despues, al volver a clases de nuevo nos encontramos y quedamos en un "acuerdo" donde nadie podría salir con nadie por el bien de la "relación" eso me hacía sentir incomodo porque es lo que ma detestaba en alguien, pretender que solo con un "acuerdo" solucionas las cosas, no pude decir nada porque su actitud hacia alguien que no cumple sus caprichos es algo muy inmaduro. Ella dijo que en ese tiempo va a cambiar de actitud, pero no fue asi, la mayoria del tiempo se pasaba diciendo tonterías al frente mio, cosas que en cierta cosa me hieren, pero como ya no eramos novios lo dejé pasar.
  
En 1 mes tuvimos una pequeña discusión solo porque me acerque a saludar a unas amigas, yo no vi el problema pero ella buscó hasta los likes en cada foto de mis amistades en FB, hasta que me hizo problema de que se sentia insegura y del porque debo estar reaccionando a las fotos de otras chicas, mi respuesta solo fue: ya no estamos en una relación, asi que puedo hacerlo. Ella solo empezó a llorar y decir que no es lo que ella quiere en un "acuerdo". Así pasamos hablando durante 1 hora, donde ella me atacaba en cierta forma y yo solo me quedaba callado hasta que se me salio de un problema que tuve gracias a ella donde involucró antidepresivos por lo mal que me solia hacer sentir cuando se desquitaba confmigo. Por qué no dije hasta ahi quedan las cosas y ya no hay acuerdo? no se en que estaba pensando, creo porque aun sentia algo por ella. Poco a poco me alejaba de ella por mi paz mental, pero no era suficiente
  
3 meses después pasó lo mismo, pero en este caso fue un comentario que le hice a una chica, fue lo mismo que la anterior vez, tachandome de infiel y que estoy rompiendo su "acuerdo", maldita sea hubiera dicho que ya no quiero seguir con eso y alejarme de ella.
  
Me juntaba muy bien con su grupo de amigos, al tal punto de compartir algunas cosas privadas mias, ahi fue mi error. Conocí a una chica por internet y nos llevabamos bien, era de otra ciudad y bueno creo que nos empezamos a gustar, esta chica quería venir a la ciudad donde vivo, la verdad no sabia a donde llevarla porque no tenia dinero, nuestras intenciones no solo era pasear, si no de hacer otras cosas. Le dije a uno de ese grupo de amigos que lo llamaremos H, que si no podría prestarme un cuarto, pero dijo que no.
  
3 semanas despues tuvimos que hacer un proyecto de una materia donde estabamos su grupo de amigos haciendo y ella habia faltado por asuntos de trabajo, salimos a comprar con un amigo que me junto muy bien y al regresar este chico al que le dije si podia prestarme un cuarto, nos salio con la noticia que le iban a sacar del grupo por no venir, este se le comento a la chica y se armó una pelea entre ella y nosotros. Le explique que la idea no fue mia, que a mi solo me avisaron. Al dia siguiente ella asistio para realizar este trabajo y si tocaba comprar algo ella debia poner el dinero que no pudo dar el dia anterior, salio con este chico H, este le habia dicho que yo era el cabecilla de sacarla del grupo. Cuando terminamos y fuimos a tomar algo, ella me confrontó diciendo que porque tuve esa idea, los demas chicos se quedaron con cara de sorprendidos, le dije que no que fue este chico H, en fin, este chico es muy inmaduro para su edad y el dia siguiente le mostró el chat donde le habia preguntado eso.
  
Ella despues de clases me confrontó diciendome que quien era, le expliqué y a lo cual ella procedio a agredirme, es la segunda vez que lo hacía, en medio de los golpes, aruñasos, etc. me nacía ese deseo de devolverle un golpe, pero por razones divinas me contuve, sali corriendo de ahi y por suerte me encontré con un ingeniero a lo cual le empecé a hacer la conversa, ella estuvo todo el tiempo ahi esperando, al acabar la conversa me llevo a otro lugar donde me dijo que soy una mierda de persona, que es la primera vez que alguien le hace sentir asi, que espera que me vaya mal en todo y cosas asi.
  
Lo unico que le dije fue: Yo te fui sincero y honesto, nunca te fallé durante la realción, estoy orgulloso de eso, de aqui a lo que yo haga cuando tu terminaste conmigo es problema mio. Ella lloró, pataleó, poniendose en un lugar donde el puesto le quedaba grande, el sentirse madura, el sentirse que siempre ella tiene la razon. Yo solo tomaba distancia para que no me pegue. Terminó su discurso, fui a casa, le conté todo a mi madre, lloré un poco y ella me calmó diciendo que lo que dije estuvo bien, no tengo responsabilidad con ella despues de haber terminado. me dijo qye podia hacer lo que se me diera la gana.
  
El siguiente dia siguiente fui a la universidad, no saludé con sus amigos porque me dijo que ya no quiere que me junte con ellos, me senté en la aprte de atras, 2 de ellos se acercaron y me dijeron que ella no es quien para obligarlos a juntarse o no conmigo, que seguiriamos siendo amigos y no importa que cosas diga ella de mi, no cambiaran de opinion y que ella al agredirme estruvo demasiado mal y el contar orgullosamente de que me alzo la mano estuvo peor. Juro que quería llorar porque al fin alguien esta de mi lado y me entiende. Me dijeron que el chico H le contó todo porque le dije que el tuvo la idea de sacarla del grupo de trabajo y que le valia la "amistad que teniamos", me enfadé mucho con este chico a lo cual solo les dije que si me viene a molestar o algo que en verdad me enfade me desquitaria de todo lo que lla me dijo
  
Hoy fue la exposicion de dicho trabajo, ella lanzaba comentarios a diestra y siniestra de mi, solo me reía hasta que le dije a uno de los chicos que me vino a ver ese día, le dije que si seguia diciendo cosas de mi, el chico H no saldría bien ese día.
  
saludé con mis amigas del otro curso, prestandole cosas, riendo, bromeando, tal vez un comentario picarón. al finalizar la epxosicion fuimos a tomar algo, ella al despedirse, enojada me dice que que onda con las "amiguitas", que la respete, mi respuesta solo fue el decir: respetar que, tu misma lo dijiste.
  
Solo se que si de nuevo empieza a lanzar comentarios que me ofendan, actos que me afecten o violencia fisica, me desquitaría con este chico H, no por avisarla sobre lo que le dije, si no por lo hipócrita que es conmigo, con ella y con sus amigos.
"""


In [22]:
print(f"Initial string:  {len(s.split())}")

Initial string:  1462

In [25]:
text = s.lower()
# remove numbers
text = re.sub(r"\b\d+\b", '', text)
text = re.sub("  ", ' ', text)
print(f"Number of words:  {len(s.split())}")

Number of words:  1462

In [26]:
text = ' '.join([word for word in s.split() if word not in stopwords])

In [27]:
# remove numbers
text = re.sub(r"\b\d+\b", '', text)
text = re.sub("  ", ' ', text)
text = textstat.remove_punctuation(text)

# print(text)

In [28]:
# text = textstat.remove_punctuation(text)
print(f"Not empty words:  {len(text.split())}")

Not empty words:  528

In [29]:
# Split the text into words

# Create a Counter of the words
word_counter = Counter(text.split())

print(f"Distinct words  : {len(word_counter)}")
# print(word_counter)

Distinct words  : 363